In [ ]:
!pip install hvplot
!pip install jupyter_bokeh
!pip install panel
!pip install bokeh
!pip install holoviews

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!gdown 'https://drive.google.com/uc?export=download&id=1L42Fjkh8-cPIwswMHK5GtyecYdF-_Xjz'

df = pd.read_csv('owid-covid-data.csv')

Downloading...
From: https://drive.google.com/uc?export=download&id=1L42Fjkh8-cPIwswMHK5GtyecYdF-_Xjz
To: /content/owid-covid-data.csv
100% 93.9M/93.9M [00:01<00:00, 59.1MB/s]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets

!gdown 'https://drive.google.com/uc?export=download&id=1L42Fjkh8-cPIwswMHK5GtyecYdF-_Xjz'

df = pd.read_csv('owid-covid-data.csv')

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])
df
df = df.dropna(subset=['continent'])
print(df['continent'].unique())

# Ensure 'date' column is in datetime format
# df.loc[:, 'date'] = pd.to_datetime(df['date'])

# Extract unique values for continents and countries
continents = df['continent'].dropna().unique()  # Handle potential NaN values
countries = df['location'].unique()
years = [2020, 2021, 2022, 2023]  # Limit years for simplicity

# Create a function to filter countries based on the selected continent
def get_countries_by_continent(continent):
    if continent:
        return df[df['continent'] == continent]['location'].unique()
    else:
        return []

# Define the plotting function
def plot_data(continent, country, year):
    # Filter the data based on user selection
    filtered_data = df[(df['continent'] == continent) &
                       (df['location'] == country) &
                       (df['date'].dt.year == year)]

    # Check if there's data to plot
    if filtered_data.empty:
        print(f"No data available for {country} in {year}.")
        return

    # Set up the figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Plot 1: Total Cases Over Time
    axes[0, 0].plot(filtered_data['date'], filtered_data['total_cases'], label='Total Cases', color='blue', linewidth=2)
    axes[0, 0].set_title(f'Total Cases in {country} ({year})', fontsize=14)
    axes[0, 0].set_xlabel('Date', fontsize=12)
    axes[0, 0].set_ylabel('Total Cases', fontsize=12)
    axes[0, 0].grid(True, linestyle='--', alpha=0.6)

    # Plot 2: Total Deaths Over Time
    axes[0, 1].plot(filtered_data['date'], filtered_data['total_deaths'], label='Total Deaths', color='red', linewidth=2)
    axes[0, 1].set_title(f'Total Deaths in {country} ({year})', fontsize=14)
    axes[0, 1].set_xlabel('Date', fontsize=12)
    axes[0, 1].set_ylabel('Total Deaths', fontsize=12)
    axes[0, 1].grid(True, linestyle='--', alpha=0.6)

    # Plot 3: Total Vaccinations Over Time
    axes[1, 0].plot(filtered_data['date'], filtered_data['total_vaccinations'], label='Total Vaccinations', color='green', linewidth=2)
    axes[1, 0].set_title(f'Total Vaccinations in {country} ({year})', fontsize=14)
    axes[1, 0].set_xlabel('Date', fontsize=12)
    axes[1, 0].set_ylabel('Total Vaccinations', fontsize=12)
    axes[1, 0].grid(True, linestyle='--', alpha=0.6)

    # Plot 4: New Cases Over Time
    axes[1, 1].plot(filtered_data['date'], filtered_data['new_cases'], label='New Cases', color='orange', linewidth=2)
    axes[1, 1].set_title(f'New Cases in {country} ({year})', fontsize=14)
    axes[1, 1].set_xlabel('Date', fontsize=12)
    axes[1, 1].set_ylabel('New Cases', fontsize=12)
    axes[1, 1].grid(True, linestyle='--', alpha=0.6)

    # Add a main title
    plt.suptitle('Covid19 Dashboard ~ Hanita', fontsize=20, fontweight='bold', y=1.02)

    # Adjust layout
    for ax in axes.flat:
        ax.legend(fontsize=10)
        ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.show()

# Create interactive widgets for continent and year
continent_widget = widgets.Dropdown(options=continents, description='Continent:')
year_widget = widgets.IntSlider(
    min=2020,
    max=2023,
    step=1,
    value=2020,
    description='Year:',
    continuous_update=False
)

# Create the country widget with initial options based on the selected continent
country_widget = widgets.Dropdown(description='Country:')

# Update the countries dropdown when a continent is selected
def update_country_options(continent):
    country_widget.options = get_countries_by_continent(continent)

# Link the continent widget to the update function
continent_widget.observe(lambda change: update_country_options(change.new), names='value')

# Set initial country options
update_country_options(continent_widget.value)

# Create the interactive plot
interact(
    plot_data,
    continent=continent_widget,
    country=country_widget,
    year=year_widget
);

Downloading...
From: https://drive.google.com/uc?export=download&id=1L42Fjkh8-cPIwswMHK5GtyecYdF-_Xjz
To: /content/owid-covid-data.csv
100% 93.9M/93.9M [00:00<00:00, 235MB/s]
['Asia' 'Europe' 'Africa' 'Oceania' 'North America' 'South America']


interactive(children=(Dropdown(description='Continent:', options=('Asia', 'Europe', 'Africa', 'Oceania', 'Nort…